In [1]:
pip install cryptography


In [3]:
import json
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, hashes, kdf
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
import base64

# Encryption key and IV setup
PASSWORD = b'Aaron'  # Change this password
SALT = os.urandom(16)

# Key derivation
def derive_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password)

# Encrypt the JSON data
def encrypt_data(data, password, salt):
    key = derive_key(password, salt)
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # JSON to string, then bytes
    padded_data = pad_data(json.dumps(data).encode('utf-8'))
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    return {
        'salt': base64.b64encode(salt).decode('utf-8'),
        'iv': base64.b64encode(iv).decode('utf-8'),
        'data': base64.b64encode(encrypted_data).decode('utf-8')
    }

# Decrypt the JSON data
def decrypt_data(encrypted_data, password):
    salt = base64.b64decode(encrypted_data['salt'])
    iv = base64.b64decode(encrypted_data['iv'])
    data = base64.b64decode(encrypted_data['data'])

    key = derive_key(password, salt)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    padded_data = decryptor.update(data) + decryptor.finalize()
    json_data = unpad_data(padded_data).decode('utf-8')

    return json.loads(json_data)

# Padding for AES block size (128 bits)
def pad_data(data):
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    return padder.update(data) + padder.finalize()

def unpad_data(data):
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    return unpadder.update(data) + unpadder.finalize()

# Keyword search in decrypted data
def search_keywords(decrypted_data, keywords):
    matched_records = []
    for record in decrypted_data.get("records", []):
        if any(keyword.lower() in record.get("condition", "").lower() for keyword in keywords):
            matched_records.append(record)
    return matched_records

# Example usage
if __name__ == "__main__":
    # Sample medical records JSON data
    medical_data = {
        "records": [
            {"id": 1, "name": "John Doe", "age": 50, "condition": "Hypertension"},
            {"id": 2, "name": "Jane Smith", "age": 30, "condition": "Diabetes"},
            {"id": 3, "name": "Mike Lee", "age": 40, "condition": "Hypertension"},
            {"id": 4, "name": "Emily Davis", "age": 35, "condition": "Healthy"},
        ]
    }

    # Encrypt the data
    encrypted_data = encrypt_data(medical_data, PASSWORD, SALT)
    print("Encrypted Data:", encrypted_data)

    # Decrypt the data and search for a keyword
    decrypted_data = decrypt_data(encrypted_data, PASSWORD)
    keywords = ["hypertension"]
    results = search_keywords(decrypted_data, keywords)
    print("\nFiltered Records with keyword(s):", keywords)
    print(results)


Encrypted Data: {'salt': '0VXKi/Bc9/ijX4JHcz5MJA==', 'iv': 'EkvWk+ojV0OVNig76IL5oQ==', 'data': 'gK6jBuZUAc7AeJTfJGaDVjHe+1EIJbEN6AYIr6s7XXHCSuSTFHfUNYXXp1RDo3yFOoqaoB/P4OEUeWAfYn3r+S0d24kz0TZUstoH2KwL/Y6JzQgrGLmBgG94w2Lr8ltkY5QBFbJLWqX2kitrjgkuD1RVPjU/IWUFM8/9vNU2Aq6Pqw5HR+1iVae+IwxNj3ST7TjR9J/XbKYYfEXHEHwsgj2CeLf4QY+smmOPYnOAK7ld32VuTHQdaGy7zzNlKYHlPdzlMPkUL520/WQGItokVHfgAzlRiTshccxXvBAj+BjkamvoX1XcKss5hD0pnPVrtBolKOGRrpdvXoRuvhwPS/dEaoW/HshFmwRh8QxwXFgL2918JU4u+ZYEC2YbUJ5ahPLkP7mkb/sXcLROXmQyJw=='}

Filtered Records with keyword(s): ['hypertension']
[{'id': 1, 'name': 'John Doe', 'age': 50, 'condition': 'Hypertension'}, {'id': 3, 'name': 'Mike Lee', 'age': 40, 'condition': 'Hypertension'}]


In [8]:
import json
import base64
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import hashlib

# Set a secure password and salt for AES encryption
PASSWORD = b'Aaron'  # Change this password
SALT = os.urandom(16)

# Key derivation function
def derive_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password)

# Encrypt a data field
def encrypt_field(data, password, salt):
    key = derive_key(password, salt)
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    padded_data = pad_data(data.encode('utf-8'))
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    return {
        'salt': base64.b64encode(salt).decode('utf-8'),
        'iv': base64.b64encode(iv).decode('utf-8'),
        'data': base64.b64encode(encrypted_data).decode('utf-8')
    }

# Decrypt a data field
def decrypt_field(encrypted_data, password):
    salt = base64.b64decode(encrypted_data['salt'])
    iv = base64.b64decode(encrypted_data['iv'])
    data = base64.b64decode(encrypted_data['data'])

    key = derive_key(password, salt)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    padded_data = decryptor.update(data) + decryptor.finalize()
    return unpad_data(padded_data).decode('utf-8')

# Padding function for AES block size
def pad_data(data):
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    return padder.update(data) + padder.finalize()

# Unpadding function
def unpad_data(data):
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    return unpadder.update(data) + unpadder.finalize()

# Hash the keyword for secure search index
def hash_keyword(keyword):
    return hashlib.sha256(keyword.encode()).hexdigest()

# Encrypt and store medical data with hashed keywords for search indexing
def encrypt_and_store_medical_data(medical_data, password):
    encrypted_records = []
    for record in medical_data:
        # Encrypt sensitive fields only
        encrypted_record = {
            "id": record["id"],
            "name": record["name"],
            "age": record["age"],
            "condition": record["condition"],
            "medicine_info": encrypt_field(record["medicine_info"], password, SALT),
            "past_medical_history": encrypt_field(record["past_medical_history"], password, SALT),
            "keywords": [hash_keyword(keyword) for keyword in record["condition"].split()]
        }
        encrypted_records.append(encrypted_record)
    return encrypted_records

# Search and decrypt specific fields based on hashed keyword
def search_and_retrieve_records(encrypted_records, keyword, password):
    keyword_hash = hash_keyword(keyword)
    results = []

    for record in encrypted_records:
        if keyword_hash in record["keywords"]:
            decrypted_record = {
                "id": record["id"],
                # "name": record["name"],
                "condition": record["condition"],
                "medicine_info": decrypt_field(record["medicine_info"], password),
                "past_medical_history": decrypt_field(record["past_medical_history"], password)
            }
            results.append(decrypted_record)
    return results

# Example usage
if __name__ == "__main__":
    # Sample medical records
    medical_data = [
        {
            "id": 1,
            "name": "John Doe",
            "age": 50,
            "condition": "Hypertension Diabetes",
            "medicine_info": "Lisinopril, Metformin",
            "past_medical_history": "High blood pressure since 2010, Type 2 diabetes"
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "age": 30,
            "condition": "Asthma",
            "medicine_info": "Albuterol",
            "past_medical_history": "Chronic respiratory issues since childhood"
        }
    ]

    # Encrypt and store the data
    encrypted_records = encrypt_and_store_medical_data(medical_data, PASSWORD)
    print("Encrypted Records for Database Storage:\n", json.dumps(encrypted_records, indent=2))

    # Search and retrieve based on keyword
    search_keyword = "Hypertension"
    results = search_and_retrieve_records(encrypted_records, search_keyword, PASSWORD)
    print("\nDecrypted Records with keyword '{}':\n".format(search_keyword), json.dumps(results, indent=2))


Encrypted Records for Database Storage:
 [
  {
    "id": 1,
    "name": "John Doe",
    "age": 50,
    "condition": "Hypertension Diabetes",
    "medicine_info": {
      "salt": "Lyqxe5JMYJAPVVBOa1afwA==",
      "iv": "GStdr8S1KjBfP+qEc8TTgQ==",
      "data": "+ABj5bt/6rfdMPMaml4BbDIW3ZGZAC1tTr8jkRhZnh0="
    },
    "past_medical_history": {
      "salt": "Lyqxe5JMYJAPVVBOa1afwA==",
      "iv": "23MDBzW7iGQZ6VZZ/hMjcQ==",
      "data": "FwWtdDjlTCxuq2vtP/dB9r2IDychUnD3pFzmR0CAwUUrzEgGGj1poK5GACpIWvfQ"
    },
    "keywords": [
      "cc5cc8d5ca1325a4a526118580a3a27b1f3821c734dcc9289b3a67e5d7ef027c",
      "c3311c4ca6df58c509e76fba2049d3d2e8c7256cd84fa19d2c3ccda6b372fb6d"
    ]
  },
  {
    "id": 2,
    "name": "Jane Smith",
    "age": 30,
    "condition": "Asthma",
    "medicine_info": {
      "salt": "Lyqxe5JMYJAPVVBOa1afwA==",
      "iv": "VBNbhifqfwZQUbWh/8c7Iw==",
      "data": "nV5k5bSdsp6WvL7BD8JAGg=="
    },
    "past_medical_history": {
      "salt": "Lyqxe5JMYJAPVVBOa1afwA==",


In [14]:
import json
import base64
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import hashlib

# Set a secure password and salt for AES encryption
PASSWORD = b'Aaron'  # Change this password
SALT = os.urandom(16)

# Key derivation function
def derive_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password)

# Encrypt a data field
def encrypt_field(data, password, salt):
    key = derive_key(password, salt)
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    padded_data = pad_data(data.encode('utf-8'))
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    return {
        'salt': base64.b64encode(salt).decode('utf-8'),
        'iv': base64.b64encode(iv).decode('utf-8'),
        'data': base64.b64encode(encrypted_data).decode('utf-8')
    }

# Decrypt a data field
def decrypt_field(encrypted_data, password):
    salt = base64.b64decode(encrypted_data['salt'])
    iv = base64.b64decode(encrypted_data['iv'])
    data = base64.b64decode(encrypted_data['data'])

    key = derive_key(password, salt)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    padded_data = decryptor.update(data) + decryptor.finalize()
    return unpad_data(padded_data).decode('utf-8')

# Padding function for AES block size
def pad_data(data):
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    return padder.update(data) + padder.finalize()

# Unpadding function
def unpad_data(data):
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    return unpadder.update(data) + unpadder.finalize()

# Hash the keyword for secure search index
def hash_keyword(keyword):
    return hashlib.sha256(keyword.encode()).hexdigest()

# Encrypt and store medical data with hashed keywords for search indexing
def encrypt_and_store_medical_data(medical_data, password):
    encrypted_records = []
    for record in medical_data:
        # Encrypt sensitive fields only
        encrypted_record = {
            "id": record["id"],
            "name": record["name"],
            "age": record["age"],
            "condition": record["condition"],
            "medicine_info": encrypt_field(record["medicine_info"], password, SALT),
            "past_medical_history": encrypt_field(record["past_medical_history"], password, SALT),
            "keywords": [hash_keyword(keyword) for keyword in record["condition"].split()]
        }
        encrypted_records.append(encrypted_record)
    return encrypted_records

# # Search and decrypt specific fields based on hashed keyword
# def search_and_retrieve_records(encrypted_records, keyword, password):
#     keyword_hash = hash_keyword(keyword)
#     results = []

#     for record in encrypted_records:
#         if keyword_hash in record["keywords"]:
#             decrypted_record = {
#                 "id": record["id"],
#                 # "name": record["name"],
#                 "condition": record["condition"],
#                 "medicine_info": decrypt_field(record["medicine_info"], password),
#                 "past_medical_history": decrypt_field(record["past_medical_history"], password)
#             }
#             results.append(decrypted_record)
#     return results

def search_and_retrieve_records(encrypted_records, keyword, password):
    keyword_hash = hash_keyword(keyword)
    results = []

    for record in encrypted_records:
        if keyword_hash in record["keywords"]:
            decrypted_record = {
                "id": record["id"],
                "condition": record["condition"],
                "medicine_info": decrypt_field(record["medicine_info"], password),
                "past_medical_history": decrypt_field(record["past_medical_history"], password)
            }
            results.append(decrypted_record)
    return results


# # Example usage
# if __name__ == "__main__":
#     # Sample medical records
#     medical_data = [
#         {
#             "id": 1,
#             "name": "John Doe",
#             "age": 50,
#             "condition": "Hypertension Diabetes",
#             "medicine_info": "Lisinopril, Metformin",
#             "past_medical_history": "High blood pressure since 2010, Type 2 diabetes"
#         },
#         {
#             "id": 2,
#             "name": "Jane Smith",
#             "age": 30,
#             "condition": "Asthma",
#             "medicine_info": "Albuterol",
#             "past_medical_history": "Chronic respiratory issues since childhood"
#         }
#     ]

#     # Encrypt and store the data
#     encrypted_records = encrypt_and_store_medical_data(medical_data, PASSWORD)
#     print("Encrypted Records for Database Storage:\n", json.dumps(encrypted_records, indent=2))

#     # Search and retrieve based on keyword
#     search_keyword = "Hypertension"
#     results = search_and_retrieve_records(encrypted_records, search_keyword, PASSWORD)
#     print("\nDecrypted Records with keyword '{}':\n".format(search_keyword), json.dumps(results, indent=2))

import getpass

if __name__ == "__main__":
    # Sample medical records
    medical_data = [
        {
            "id": 1,
            "name": "John Doe",
            "age": 50,
            "condition": "Hypertension Diabetes",
            "medicine_info": "Lisinopril, Metformin",
            "past_medical_history": "High blood pressure since 2010, Type 2 diabetes"
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "age": 30,
            "condition": "Asthma",
            "medicine_info": "Albuterol",
            "past_medical_history": "Chronic respiratory issues since childhood"
        }
    ]

    # Encrypt and store the data
    encrypted_records = encrypt_and_store_medical_data(medical_data, PASSWORD)
    print("Data successfully encrypted and stored.")

    # Prompt the user for password input
    user_password = getpass.getpass("Enter the decryption password: ").encode('utf-8')

    # Check if the provided password matches
    if user_password == PASSWORD:
        # Prompt the user for keyword input
        search_keyword = input("Enter the medical condition keyword to search: ").strip()

        # Search and retrieve records based on user input
        results = search_and_retrieve_records(encrypted_records, search_keyword, user_password)

        # Display results
        if results:
            print("\nDecrypted Records matching '{}':\n".format(search_keyword), json.dumps(results, indent=2))
        else:
            print("No records found for the specified keyword.")
    else:
        print("Incorrect password. Access denied.")



Data successfully encrypted and stored.
Enter the decryption password: ··········
Enter the medical condition keyword to search: Asthma

Decrypted Records matching 'Asthma':
 [
  {
    "id": 2,
    "condition": "Asthma",
    "medicine_info": "Albuterol",
    "past_medical_history": "Chronic respiratory issues since childhood"
  }
]
